# Part 1: Generate Hamiltonian, and Partition it

In [210]:
from pert_trotter.ham_utils import obtain_OF_hamiltonian

from openfermion import (
    count_qubits,
    jordan_wigner,
)

from pert_trotter.tensor_utils import get_chem_tensors, obt2op, tbt2op

mol = [["H", [0, 0, 0]], ["H", [0, 0, 0.8]]]
H, num_elecs = obtain_OF_hamiltonian(mol)
n_qubits = count_qubits(H)
assert n_qubits == 4

In [211]:
from openfermion.linalg import qubit_operator_sparse

H_const, H_obt, H_tbt = get_chem_tensors(H=H, N=n_qubits)
H_ob_op = obt2op(H_obt)
H_tb_op = tbt2op(H_tbt)
H_ele = H_const + H_ob_op + H_tb_op

In [212]:
import pandas as pd
from dataclasses import dataclass, asdict
from openfermion import number_operator
import scipy as sp
import numpy as np
jw_of = jordan_wigner(H_ele)
jw_of_sp = qubit_operator_sparse(jw_of)
jw_op_array = jw_of_sp.toarray()
eigenvalues, eigenvectors = sp.linalg.eigh(jw_op_array)
eigenvalue_0 = eigenvalues[0]
eigenvectors_0 = eigenvectors[:, [0]]
eigenvectors_0_sparse = sp.sparse.csc_matrix(eigenvectors_0)

def get_on_num(w, e: int) -> float:
    on_op = number_operator(n_modes=e, parity=-1)
    on_op_sparse = qubit_operator_sparse(jordan_wigner(on_op))
    b = on_op_sparse * w
    n = np.divide(b, w)
    n = n[~np.isnan(n)]
    n = n[0]
    if np.array_equal(b, n * w):
        return n
    else:
        raise UserWarning("may not have returned correct number of electrons")

@dataclass
class EnergyOccupation:
    energy: float
    spin_orbs: int

def dc_to_dict(dcs, labels: list[str]):
    attrs = list(dcs[0].__dict__.keys())
    tb_dict = {}
    for i, label in enumerate(labels):
        tb_dict[label] = []
        for dc in dcs:
            tb_dict[label].append(getattr(dc, attrs[i]))
    return tb_dict

tb = []
for i in range(eigenvectors.shape[0]):
    w = eigenvectors[:, [i]]
    n = get_on_num(w, 4)
    tb.append(EnergyOccupation(energy=eigenvalues[i], spin_orbs=n))

pd.DataFrame.from_dict(dc_to_dict(tb, labels=["Energy (Hartree)", "Occupied Spin Orbitals"]))

/var/folders/_g/6lsfhb794rzg7hm54031db2r0000gn/T/ipykernel_59284/1462428053.py:18: RuntimeWarning: invalid value encountered in divide
  n = np.divide(b, w)


,Energy (Hartree),Occupied Spin Orbitals
0,-1.134148,2.0+0.0j
1,-0.597178,2.0+0.0j
2,-0.597178,2.0+0.0j
3,-0.597178,2.0+0.0j
4,-0.556355,1.0+0.0j
5,-0.556355,1.0-0.0j
6,-0.498232,3.0+0.0j
7,-0.498232,3.0+0.0j
8,-0.227924,2.0-0.0j
9,0.151834,1.0+0.0j


In [ ]:
# with projection into symmetry subspace of ground site Hamiltonian

from openfermion import (
    s_squared_operator,
    get_number_preserving_sparse_operator,
)

project = False
projector_func = None

if project:
    s_sq = s_squared_operator(n_qubits // 2)
    s_sq_sparse = get_number_preserving_sparse_operator(
        s_sq, n_qubits, num_elecs, spin_preserving=True
    )
    s_sq_array = s_sq_sparse.toarray()
    s_sq_values, s_sq_vectors = np.linalg.eigh(s_sq_array)
    s_sq_vectors_sparse = sp.sparse.csc_matrix(s_sq_vectors)
    non_cisd_dim = len(list(filter(lambda n: n <= 0.01, s_sq_values)))
    s_sq_evals, nsz2ssq_proj = (
        s_sq_values[:non_cisd_dim],
        s_sq_vectors[:, :non_cisd_dim].T,
    )
    nsz2ssq_proj_sparse = sp.sparse.csc_matrix(nsz2ssq_proj)
    projector_func = lambda f, excitation_level: get_projected_sparse_op(
        H_OF=f,
        NSz2SSq_Proj_sparse=nsz2ssq_proj_sparse,
        n_qubits=n_qubits,
        num_elecs=num_elecs,
        excitation_level=excitation_level,
    )

In [213]:
from typing import Optional
from pert_trotter.ffrag_utils import LR_frags_generator
from openfermion import FermionOperator

# without projection into any subspace (???)
def do_lr_fo(H_FO: FermionOperator, projector_func: Optional, project: bool = False,):
    const, obt, tbt = get_chem_tensors(H_FO)
    obt_op = obt2op(obt)

    # Obtaining LR fragments as list of FermionOperators and (coeffs, angles) defining the fragments.
    lowrank_fragments, lowrank_params = LR_frags_generator(
        tbt, tol=1e-5, ret_params=True
    )

    # Filtering out small fragments
    LR_fragments = []
    LR_params = []
    for i in range(len(lowrank_params)):
        frag = lowrank_fragments[i]
        if frag.induced_norm(2) > 1e-6:
            LR_fragments.append(frag)
            LR_params.append(lowrank_params[i])
    all_frag_ops = [const * FermionOperator.identity(), obt_op]
    all_frag_ops += LR_fragments

    if project:
        pass

    return const * FermionOperator.identity(), obt_op, LR_fragments

def sum_partitions(frags, e: int):
    min_e_each_frag = []
    for frag in frags:
        values, wectors = sp.linalg.eigh(frag.toarray())
        allowed_values = []
        print(wectors.shape)
        for i in range(wectors.shape[1]):
            w = wectors[:, [i]]
            n = get_on_num(w, 4)
            if n == 2:
                allowed_values.append(eigenvalues[i])
        min_e_each_frag.append(min(allowed_values))
    return sum(min_e_each_frag)

const, H1, H2_frags = do_lr_fo(H_ele)
dfs = []
allowed_energies = []
all_energies = []
for frag in H2_frags:
    jw_of = jordan_wigner(frag)
    jw_of_sp = qubit_operator_sparse(jw_of)
    jw_op_array = jw_of_sp.toarray()
    eigenvalues, eigenvectors = sp.linalg.eigh(jw_op_array)
    tb = []
    energies = []
    all_en = []
    for i in range(eigenvectors.shape[0]):
        energy = eigenvalues[i]
        w = eigenvectors[:, [i]]
        n = get_on_num(w, 4)
        tb.append(EnergyOccupation(energy=energy, spin_orbs=n))
        all_en.append(energy)
        if n == 2:
            energies.append(energy)
    all_energies.append(all_en)
    allowed_energies.append(energies)
    dfs.append(pd.DataFrame.from_dict(dc_to_dict(tb, labels=["Energy (Hartree)", "Occupied Spin Orbitals"])))

[[1.7329818531360702e-06, 1.7329818531360702e-06, 1.7329818531412744e-06, 1.7329818531412744e-06, 0.02112801853383596, 0.021900346805237128], [0.0, 6.938893903907228e-18, 2.220446049250313e-16, 2.220446049250313e-16, 0.36925356711212226, 0.36925356711212226], [1.3047599509219954, 1.3285004816595831, 1.3285004816595831, 1.328500481659584, 1.328500481659585, 1.3524550528490995]]


/var/folders/_g/6lsfhb794rzg7hm54031db2r0000gn/T/ipykernel_59284/1462428053.py:18: RuntimeWarning: invalid value encountered in divide
  n = np.divide(b, w)


In [214]:
dfs[0]

,Energy (Hartree),Occupied Spin Orbitals
0,-8.673617e-19,0.0+0.0j
1,1.732982e-06,2.0+0.0j
2,1.732982e-06,2.0+0.0j
3,1.732982e-06,2.0+0.0j
4,1.732982e-06,2.0+0.0j
5,6.931927e-06,4.0+0.0j
6,5.092389e-03,3.0+0.0j
7,5.092389e-03,3.0+0.0j
8,5.282005e-03,1.0+0.0j
9,5.282005e-03,1.0+0.0j


In [215]:
dfs[1]

,Energy (Hartree),Occupied Spin Orbitals
0,0.000000e+00,0.0+0.0j
1,0.000000e+00,2.0+0.0j
2,6.938894e-18,2.0+0.0j
3,6.938894e-18,4.0+0.0j
4,2.220446e-16,2.0+0.0j
5,2.220446e-16,2.0+0.0j
6,9.231339e-02,1.0+0.0j
7,9.231339e-02,3.0+0.0j
8,9.231339e-02,1.0+0.0j
9,9.231339e-02,3.0+0.0j


In [216]:
dfs[2]

,Energy (Hartree),Occupied Spin Orbitals
0,-2.775558e-16,0.0+0.0j
1,3.261900e-01,1.0+0.0j
2,3.261900e-01,1.0+0.0j
3,3.381138e-01,1.0+0.0j
4,3.381138e-01,1.0+0.0j
5,1.304760e+00,2.0+0.0j
6,1.328500e+00,2.0+0.0j
7,1.328500e+00,2.0+0.0j
8,1.328500e+00,2.0+0.0j
9,1.328500e+00,2.0+0.0j


In [219]:
H_no_two_body = const + H1
jw_of = jordan_wigner(H_no_two_body)
jw_of_sp = qubit_operator_sparse(jw_of)
jw_op_array = jw_of_sp.toarray()
eigenvalues, eigenvectors = sp.linalg.eigh(jw_op_array)
energy_no_two_body = eigenvalues[0]
eigenvectors_0 = eigenvectors[:, [0]]
eigenvectors_0_sparse = sp.sparse.csc_matrix(eigenvectors_0)
two_body_contributions = sum([min(e) for e in allowed_energies])
two_body_contributions_not_filtered = sum([min(e) for e in all_energies])
print(f"Two body contribution with only two occupied spin orbitals states: {two_body_contributions} Hartree")
print(f"Two body contribution with any spin orbitals states: {two_body_contributions_not_filtered} Hartree")
print(f"LR Energy using only two occupied spin orbital states: {energy_no_two_body + two_body_contributions} Hartree")
print(f"LR Energy using only any spin orbital states: {energy_no_two_body + two_body_contributions_not_filtered} Hartree")


Two body contribution with only two occupied spin orbitals states: 1.3047616839038485 Hartree
Two body contribution with any spin orbitals states: -2.7842311789427754e-16 Hartree
LR Energy using only two occupied spin orbital states: -3.2080698629765365 Hartree
LR Energy using only any spin orbital states: -4.512831546880385 Hartree


In [136]:
from pert_trotter.proj_ham import get_projected_sparse_op

projected_frags = Do_Fermi_Partitioning(
    H_ele,
    type="lr",
    tol=1e-6,
    spacial=False,
    save=False,
    load=False,
    shrink_frag=True,
    projector_func=lambda f, excitation_level: get_projected_sparse_op(
            H_OF=f,
            NSz2SSq_Proj_sparse=nsz2ssq_proj_sparse,
            n_qubits=n_qubits,
            num_elecs=num_elecs,
            excitation_level=excitation_level,
        ),
    )
summed_energies = sum_partitions(projected_frags)
print(summed_energies)

-1.3173757948560412
